# 2D Convex Hull: Graham Scan Algorithm

This notebook demonstrates the **Graham Scan** algorithm for finding the convex hull of a set of 2D points.

## Concept
The convex hull of a set of points is the smallest convex polygon that contains all the points.
Imagine stretching a rubber band around the points and letting it snap tight; the shape formed by the rubber band is the convex hull.

![convex_hull_2d](convex_hull.png)

## Algorithm: Graham Scan
Graham Scan is an efficient algorithm with time complexity $O(N \log N)$. It works by:
1.  **Start Point:** Finding the point with the lowest Y-coordinate (and lowest X if ties).
2.  **Sort:** Sorting the remaining points by the polar angle they make with the start point.
3.  **Scan:** Iterating through the sorted points and building the hull stack. If adding a new point creates a "right turn" (concave), we backtrack (pop) until a "left turn" (convex) is restored.

In [ ]:
import random
import math
import matplotlib.pyplot as plt
from compas.geometry import Point

# Helper function to plot points and lines
def plot_step(points, hull_stack=None, current_point=None, title="Convex Hull Step"):
    plt.figure(figsize=(8, 8))
    
    # Plot all points
    xs = [p.x for p in points]
    ys = [p.y for p in points]
    plt.scatter(xs, ys, color='blue', label='Points')
    
    # Plot Hull Stack
    if hull_stack:
        hx = [p.x for p in hull_stack]
        hy = [p.y for p in hull_stack]
        plt.plot(hx, hy, color='green', linestyle='-', linewidth=2, marker='o', label='Hull Stack')
        
        # Connect last point to first if complete (optional visual check)
        # plt.plot([hx[-1], hx[0]], [hy[-1], hy[0]], 'g--')

    # Plot Current Point being considered
    if current_point:
        plt.scatter([current_point.x], [current_point.y], color='red', s=100, label='Current Point')

    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

## Relevance in Architecture and Fabrication

The 3D Convex Hull has specific applications in design and construction:

*   **Collision Volumes:** Simplifying complex 3D geometries (like furniture or structural nodes) into convex shapes for faster collision detection in simulations.
    
    ![Collision Mesh Example](collision_mesh_spot_anymal.jpeg)
    *Example: Using convex hulls as simplified collision meshes for a robot (Spot).*

*   **Spatial Analysis:** Defining the "envelope" or maximum extent of a complex building form or a cluster of objects.
*   **Stock Material Estimation:** Calculating the minimal bounding volume (often tighter than a box) to estimate the raw material block needed to mill a complex shape.
*   **Robotics:** Path planning algorithms often use convex hulls of obstacles to ensure safe distances.

## 1. Generate Random Points

We generate a set of random 2D points using `compas.geometry.Point`.

In [ ]:
def generate_points(n=20, bounds=(0, 100)):
    points = []
    for _ in range(n):
        x = random.uniform(*bounds)
        y = random.uniform(*bounds)
        points.append(Point(x, y, 0))
    return points

# Generate points
points = generate_points(30)
plot_step(points, title="Generated Points")

## 2. Graham Scan Algorithm

### Step 1: Find the Start Point
Find the point with the lowest Y coordinate. If there are ties, pick the one with the lowest X.

In [ ]:
# Find the bottom-most point (and left-most if ties)
start_point = min(points, key=lambda p: (p.y, p.x))

print(f"Start Point: {start_point}")

# Visualize start point
plot_step(points, hull_stack=[start_point], title="Start Point Selected")

### Step 2: Sort Points by Polar Angle
Sort the remaining points based on the angle they make with the `start_point` and the X-axis.

In [ ]:
def polar_angle(p0, p1):
    return math.atan2(p1.y - p0.y, p1.x - p0.x)

def distance_sq(p0, p1):
    return (p1.x - p0.x)**2 + (p1.y - p0.y)**2

# Remove start point from list to sort the rest
other_points = [p for p in points if p != start_point]

# Sort by polar angle. If angles are same, sort by distance (furthest first? or closest? Graham scan usually keeps furthest)
# Actually, for Graham scan, if two points have same angle, we usually keep the furthest one and remove the others, 
# or just process them. A simple sort by angle is often sufficient for basic cases, but let's handle collinearity by distance.
other_points.sort(key=lambda p: (polar_angle(start_point, p), distance_sq(start_point, p)))

sorted_points = [start_point] + other_points

# Visualize sorted order
# We can draw lines from start point to others to show order
plt.figure(figsize=(8, 8))
plt.scatter([p.x for p in points], [p.y for p in points], c='blue')
plt.scatter([start_point.x], [start_point.y], c='red', s=100)
for i, p in enumerate(sorted_points[1:]):
    plt.text(p.x, p.y, str(i+1), fontsize=12)
    # plt.plot([start_point.x, p.x], [start_point.y, p.y], 'k--', alpha=0.3)
plt.title("Points Sorted by Polar Angle")
plt.axis('equal')
plt.show()

### Step 3: Build the Hull
Iterate through the sorted points. For each point, check if adding it creates a "left turn" (counter-clockwise). If it creates a "right turn" (clockwise), it means the previous point is not part of the convex hull, so we pop it from the stack.

In [ ]:
def cross_product(o, a, b):
    """Returns the cross product of vectors OA and OB.
    Positive means left turn, Negative means right turn, 0 means collinear.
    """
    return (a.x - o.x) * (b.y - o.y) - (a.y - o.y) * (b.x - o.x)

hull_stack = [sorted_points[0], sorted_points[1]]

# Visualize initial stack
plot_step(points, hull_stack, title="Initial Stack (First 2 points)")

for i in range(2, len(sorted_points)):
    current_point = sorted_points[i]
    
    # While the last 3 points make a non-left turn (right turn or collinear), pop the stack
    while len(hull_stack) > 1 and cross_product(hull_stack[-2], hull_stack[-1], current_point) <= 0:
        popped = hull_stack.pop()
        # Optional: Visualize the pop (backtracking)
        # plot_step(points, hull_stack + [popped], current_point, title=f"Backtracking: Popped {popped}")
    
    hull_stack.append(current_point)
    
    # Visualize progress
    # plot_step(points, hull_stack, current_point, title=f"Added Point {i}")

# Final Hull
# Add start point to close the loop for visualization
final_hull = hull_stack + [hull_stack[0]]
plot_step(points, final_hull, title="Final Convex Hull")

## 3. Using COMPAS Built-in
COMPAS also has built-in convex hull functions (usually for 3D, but works for 2D projection).

In [ ]:
from compas.geometry import convex_hull_xy, Polyline
from compas_notebook.viewer import Viewer
from compas.colors import Color

# compas.geometry.convex_hull_xy returns the coordinates of the points on the hull
# We pass [x, y] coordinates (it ignores z if provided, but let's be explicit)
hull_coords = convex_hull_xy([[p.x, p.y] for p in points])

# Ensure 3D points for Polyline (z=0)
hull_points_3d = [[pt[0], pt[1], 0.0] for pt in hull_coords]

# Create a Polyline for visualization (close the loop)
hull_polyline = Polyline(hull_points_3d + [hull_points_3d[0]])

# Visualize using compas_notebook Viewer
viewer = Viewer()
for point in points:
    viewer.scene.add(point, name="Points", pointcolor=Color.blue())
viewer.scene.add(hull_polyline, name="Convex Hull", linecolor=Color.red(), linewidth=5)
viewer.show()